# Spotify Matchmaker

Kinda like a dating app, but using your music. Scrapes data from the user's and potential match's (PM) Spotify accounts (songs, artists, audio features, etc) to create a "dating app" profile, compute their compatability score, and select artists/categories they both like. Then, the user can choose whether to "match"

In [1]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

sp = spotipy.Spotify(client_credentials_manager=SpotifyClientCredentials( client_id='30680164aa8a48eb8d6b1a6e1469f0fa', 
    client_secret='9a71c4a8a62043329a538ddcedd00e77',))

New list of "matches". IMPORTANT: ONLY RUN THIS BLOCK ONCE, or the list of matches will reset.

In [2]:
matches = []

RE-START THE PROGRAM HERE FOR EACH PM  
Enter their URI's

In [3]:
# creates lists of track ID's, titles, and artists for each person
# user
my_uri = 'spotify:user:kli-17:playlist:2T75XDoKsI4dCgPxMXz7KV' # CHANGE THIS per user
my_username = my_uri.split(':')[2]
my_playlist_id = my_uri.split(':')[4]
my_results = sp.user_playlist(my_username, my_playlist_id, 'tracks')

my_data = my_results['tracks'] 

my_titles = []
my_artists = []
my_ids = []

for track in my_data['items']:
    my_ids.append(track['track']['id'])
    my_titles.append(track['track']['name'])
    track_artist = ''
    for artist in track['track']['artists']:
        track_artist = artist['name']
    my_artists.append(track_artist)

In [4]:
# PM
their_name = "househusband" # CHANGE THIS per pm
their_uri = 'spotify:user:cranefly07:playlist:1sa38ykEGPNjjbX5ht0lZq' # CHANGE THIS per pm
their_username = their_uri.split(':')[2]
their_playlist_id = their_uri.split(':')[4]
their_results = sp.user_playlist(their_username, their_playlist_id, 'tracks')

their_data = their_results['tracks']

their_titles = []
their_artists = []
their_ids = []

for track in their_data['items']:
    their_ids.append(track['track']['id'])
    their_titles.append(track['track']['name'])
    track_artist = ''
    for artist in track['track']['artists']:
        track_artist = artist['name']
    their_artists.append(track_artist)

In [5]:
# artists both people like
shared_artists = []
for i in my_artists:
    for j in their_artists:
        if i == j and i not in shared_artists:
            shared_artists.append(i)

In [6]:
#PM's favorite artists
no_dupes = [*set(their_artists)]
counts = [] # num of occurences of each song
for i in no_dupes:
    curr_count = 0
    for j in their_artists:
        if i == j:
            curr_count = curr_count + 1
    counts.append(curr_count)
faves = []
for i in range(len(counts)):
   if counts[i] == max(counts):
       faves.append(no_dupes[i])

In [7]:
# extract audio features from each person's songs 
# create data frames (debugging purposes only)
import numpy as np
import pandas as pd

In [8]:
# user
my_features = sp.audio_features(my_ids)
my_features_df = pd.DataFrame(data=my_features, columns=my_features[0].keys())
my_features_df['title'] = my_titles
my_features_df['artist'] = my_artists
my_features_df = my_features_df[['id', 'title', 'artist', 
                           'danceability', 'energy',
                           'acousticness', 'mode', 'valence', 
                            'loudness',  'tempo'
                           ]]
my_features_df.tail()

,id,title,artist,danceability,energy,acousticness,mode,valence,loudness,tempo
60,4I8vyZU5GAVlRTzQWX6Diy,Big Black Heart,Conor Oberst,0.662,0.421,0.040000,1,0.190,-9.980,102.040
61,1170VohRSx6GwE6QDCHPPH,Kilby Girl,The Backseat Lovers,0.329,0.444,0.057800,1,0.225,-9.973,162.279
62,77G0k1La0c5Dw8bAFANcyp,Sports,Beach Bunny,0.231,0.877,0.000653,1,0.524,-5.278,186.142
63,172rW45GEnGoJUuWfm1drt,Your Best American Girl,Mitski,0.360,0.257,0.015900,1,0.130,-9.655,76.972
64,4JeOesjcWWlprTAvucDSJX,People Watching,Conan Gray,0.530,0.593,0.385000,1,0.538,-7.286,125.723


In [9]:
# repeat for PM
their_features = sp.audio_features(their_ids)
their_features_df = pd.DataFrame(data=their_features, columns=their_features[0].keys())
their_features_df['title'] = their_titles
their_features_df['artist'] = their_artists
their_features_df = their_features_df[['id', 'title', 'artist', 
                           'danceability', 'energy',
                           'acousticness', 'mode', 'valence', 
                            'loudness', 'tempo', 
                           ]]
their_features_df.tail()

,id,title,artist,danceability,energy,acousticness,mode,valence,loudness,tempo
69,5naar7XewEOAjOywIp6Jjq,Remember When,Wallows,0.504,0.708,0.00112,0,0.510,-5.896,82.009
70,4pieC53Jx9LsVLnoPSGMMk,Slow,Hauskey,0.568,0.780,0.23800,1,0.860,-4.988,199.785
71,50PyquzZ9BaBwLZZd1AuuV,Close To You,Dayglow,0.876,0.646,0.42200,1,0.832,-5.756,140.032
72,0CVaDx0YP1nMwZd1eg88cX,Water Your Garden,maye,0.573,0.728,0.14400,0,0.977,-5.656,201.974
73,4YXkB2Ti8KBX8dgUnhLoNg,F.E.A.R.,Joywave,0.713,0.588,0.00189,1,0.788,-5.063,101.007


In [10]:
# extracting traits (scale from 0 to 1, 1 is the highest amount)
def avg(lst):
    return sum(lst) / len(lst)

In [11]:
# empty list of the traits in DF
my_dance = []
my_energy = []
my_acoustic = []
my_mode = []
my_valence = []
my_loud = []
my_tempo = []

# add data from prev secion
for track in my_features:
    my_dance.append(track['danceability'])
    my_energy.append(track['energy'])
    my_acoustic.append(track['acousticness'])
    my_mode.append(track['mode'])
    my_valence.append(track['valence'])
    my_loud.append(track['loudness'])
    my_tempo.append(track['tempo'])

# scaled
my_loud_scl = 1 - (abs(avg(my_loud)) - 2) / 9 
my_tempo_scl = (avg(my_tempo) - 61) / 122

In [12]:
# repeat for PM
their_dance = []
their_energy = []
their_acoustic = []
their_mode = []
their_valence = []
their_loud = []
their_tempo = []
for track in their_features:
    their_dance.append(track['danceability'])
    their_energy.append(track['energy'])
    their_acoustic.append(track['acousticness'])
    their_mode.append(track['mode'])
    their_valence.append(track['valence'])
    their_loud.append(track['loudness'])
    their_tempo.append(track['tempo'])

# scaled
their_loud_scl = 1 - (abs(avg(their_loud)) - 2) / 9 # mean = -6.4, stdev = 2.3
their_tempo_scl = (avg(their_tempo) - 61) / 122 # mean = 123, stdev = 30.5

In [13]:
# create list of audio traits both users like, based on their avg scores for each individual characteristic
# cutoff values are +/- 0.5 stdev from the mean (using spotify's "Top 50" playlist as a benchmark)
traits = []

if avg(my_dance) > 0.57 and avg(their_dance) > 0.57:
    traits.append("danceable")
elif avg(my_dance) < 0.43 and avg(their_dance) > 0.43:
    traits.append("undanceable")
    
if avg(my_energy) > 0.58 and avg(their_energy) > 0.58:
    traits.append("energetic")
elif avg(my_energy) < 0.42 and avg(their_energy) < 0.42:
    traits.append("chill")
    
if avg(my_acoustic) > 0.62 and avg(their_acoustic) > 0.62:
    traits.append("acoustic")
elif avg(my_acoustic) < 0.38 and avg(their_acoustic) < 0.38:
    traits.append("electronically produced")
    
if avg(my_mode) > 0.7 and avg(their_mode) > 0.7:
    traits.append("written in a major key")
elif avg(my_mode) < 0.3 and avg(their_mode) < 0.3:
    traits.append("written in a minor key")
    
if avg(my_valence) > 0.72 and avg(their_valence) > 0.72:
    traits.append("happy")
elif avg(my_valence) < 0.28 and avg(their_valence) < 0.28:
    traits.append("sad")
    
if my_loud_scl > 0.64 and their_loud_scl > 0.64: 
    traits.append("loud")
elif my_loud_scl < 0.36 and their_loud_scl < 0.36:
    traits.append("quiet")

if my_tempo_scl > 0.66 and their_tempo_scl > 0.66: 
    traits.append("fast")
elif my_tempo_scl < 0.33 and their_tempo_scl < 0.33:
    traits.append("slow")
    

In [14]:
# computes compatibility scores from 0 to 1 (1 is most similar)
def get_score(lst1, lst2):
    return (1 - abs(avg(lst1) - avg(lst2)))

# individual trait
dance_score = get_score(my_dance, their_dance) 
energy_score = get_score(my_energy, their_energy)
acoustic_score = get_score(my_acoustic, their_acoustic)
mode_score = get_score(my_mode, their_mode) 
valence_score = get_score(my_valence, their_valence)
loud_score = 1 - abs(my_loud_scl - their_loud_scl)
tempo_score = 1 - abs(my_tempo_scl - their_tempo_scl)

# average of individuals
raw = (dance_score + energy_score + acoustic_score + mode_score 
       + valence_score + loud_score + tempo_score) / 7

# rounded & scaled
compatibility = round(100 * (2 * raw - 1), 3)

# if compatibility < 0 -> set to 0
if compatibility < 0: 
    compatibility = 0

In [15]:
# print info about each trait: user's avg, PM's avg, and compatbility, respectively
# print raw & scaled compatibility scores (debugging purposes only)
print("Danceability: " + str(avg(my_dance)) + " | " + str(avg(their_dance)) + " | " + str(dance_score))
print("Energy: " + str(avg(my_energy)) + " | " + str(avg(their_energy)) + " | " + str(energy_score))
print("Acousticness : " + str(avg(my_acoustic)) + " | " + str(avg(their_acoustic)) + ", " + str(acoustic_score))
print("Mode: " + str(avg(my_mode)) + " | " + str(avg(their_mode)) + " | " + str(mode_score))
print("Valence: " + str(avg(my_valence)) + " | " + str(avg(their_valence)) + " | " + str(valence_score))
print("Loudness: " + str(my_loud_scl) + " | " + str(their_loud_scl) + " | " + str(loud_score))
print("Tempo: " + str(my_tempo_scl) + " | " + str(their_tempo_scl) + " | " + str(tempo_score))
print("Raw compatibility: " + str(raw) + " | scaled compatibility: " + str(compatibility))


Danceability: 0.4888923076923078 | 0.6218918918918918 | 0.8670004158004161
Energy: 0.609476923076923 | 0.7076621621621624 | 0.9018147609147606
Acousticness : 0.21445605692307693 | 0.15076001986486487, 0.936303962941788
Mode: 0.8923076923076924 | 0.581081081081081 | 0.6887733887733887
Valence: 0.3681169230769231 | 0.6328999999999999 | 0.7352169230769232
Loudness: 0.37670085470085435 | 0.541503003003003 | 0.8351978516978513
Tempo: 0.4828609079445145 | 0.5172559813912277 | 0.9656049265532867
Raw compatibility: 0.8471303185369164 | scaled compatibility: 69.426


# Final Output 

STOP the program here!!  
This is the PM's "dating app profile"

In [16]:
# users' URI's
print("Your URI: " + my_uri)
print(their_name + "'s URI: " + their_uri)
print()

# compatibility score
if compatibility > 80:
    print("Congratulations!")
if (compatibility < 20):
    print("Uh oh :( ")
print("You and " + their_name + " are " + str(compatibility) + "% compatible.")
print()

# their favorite artists
print(their_name + "'s favorite artist(s) are: ")
print(faves)
print()

# shared artists/traits
if len(shared_artists) != 0:
    print("You both listen to: ")
    print(shared_artists)
else:
    print("You don't like any of the same artists :(")
print()

if len(traits) != 0:
    print("You both like songs that are: ")
    print(traits)
    print()

print("Do you want to match with " + their_name + "? ")

Your URI: spotify:user:kli-17:playlist:2T75XDoKsI4dCgPxMXz7KV
househusband's URI: spotify:user:cranefly07:playlist:1sa38ykEGPNjjbX5ht0lZq

You and househusband are 69.426% compatible.

househusband's favorite artist(s) are: 
['The Strokes']

You both listen to: 
['Twenty One Pilots', 'Wallows']

You both like songs that are: 
['energetic', 'electronically produced']

Do you want to match with househusband? 


ONLY UN-COMMENT THIS BLOCK TO MATCH

In [17]:
matches.append(their_name)
print("Your matches are: ")
print(matches)


Your matches are: 
['househusband']
